# Tutorial 3: Near-field basics

Learn device exposure simulations with phone antennas near the body.

## What you'll learn

- How near-field simulations work (antennas near the body)
- Antenna configuration basics
- Placement scenarios and anatomical landmarks
- Understanding the GUI and logging system
- Localized SAR extraction

**Related documentation**: [User guide (near-field workflow)](../user_guide/user_guide.md#near-field-workflow-device-exposure)

## Prerequisites

- Tutorial 1 completed (far-field basics)
- Tutorial 2 completed (config system, caching)
- Antenna CAD models available in `antenna_models/` directory

If you need antenna models, see the [Quick Start guide](../user_guide/quick_start.md) (antenna models are auto-downloaded as needed).

## About the notebook code blocks

This tutorial includes Python code blocks for running commands. These are designed for **Jupyter notebooks** if you prefer that workflow.

**If you're using notebooks**, run this setup once per session:

In [ ]:
from pathlib import Path
import importlib.util

p = Path.cwd()
while not (p / "scripts" / "notebook_helpers.py").exists():
    p = p.parent
spec = importlib.util.spec_from_file_location("_", p / "scripts" / "notebook_helpers.py")
m = importlib.util.module_from_spec(spec)
spec.loader.exec_module(m)
run_bash = m.get_run_bash()

import IPython

IPython.core.display.max_output_size = None

This helper function lets you run bash commands from Python cells using `run_bash('command')`. The setup also disables output truncation so you can see all command output.

**If you're using bash directly** (recommended), ignore the Python code blocks and just run the commands directly. Make sure to always run `source .bashrc` first.


---

## Near-field vs far-field

Far-field simulations model environmental exposure where the source is distant. Near-field simulations model device exposure where the antenna is close to the body.

The differences matter:

| Aspect | Far-field | Near-field |
|:---|:---|:---|
| Source | Plane waves (distant) | Antenna model (near body) |
| SAR pattern | Distributed | Localized hotspots |
| Configuration | Directions + polarizations | Antenna + placement |
| Typical use | Base stations, broadcast | Phones, wearables, implants |

In near-field studies, GOLIAT:
1. Imports the antenna CAD model
2. Positions it relative to anatomical landmarks
3. Applies material properties and power settings
4. Runs the FDTD simulation
5. Extracts localized SAR values

![Near-field setup](../img/tutorials/tut3_setup.png)

---

## The configuration file

This tutorial uses a single phantom (Thelonious), one frequency (700 MHz), and the front_of_eyes placement.

In [ ]:
run_bash("cat configs/tutorial_3_near_field.json")

Running: source .bashrc && cat configs/tutorial_3_near_field.json

------------------------------------------------------------
{
  "extends": "base_config.json",
  "study_type": "near_field",
  "phantoms": ["thelonious"],
  "execution_control": {
    "do_setup": true,
    "do_run": true,
    "do_extract": true
  },
  "simulation_parameters": {
    "number_of_point_sensors": 2
  },
  "gridding_parameters": {
    "global_gridding": {
      "grid_mode": "manual",
      "manual_fallback_max_step_mm": 3.0
    },
    "global_gridding_per_frequency": {
      "700": 2.5
    },
    "padding": {
      "padding_mode": "manual",
      "manual_bottom_padding_mm": [0, 0, 0],
      "manual_top_padding_mm": [0, 0, 0]
    }
  },
  "antenna_config": {
    "700": {
      "center_frequency": 700,
      "target_power_mW": 267,
      "model_type": "PIFA",
      "source_name": "Lines 1",
      "materials": {
        "Extrude 1": "Copper",
        "component1:Battery": "Copper",
        "component1:Patch": "

0



### Study type

```json
"study_type": "near_field"
```

This tells GOLIAT to run a near-field study, which requires `antenna_config` and `placement_scenarios` instead of `far_field_setup`.

### Phantom selection

```json
"phantoms": ["thelonious"]
```

We use Thelonious (male child) for this tutorial. If you don't have a license for Thelonious, feel free to use another phantoms, like Duke.

### Antenna configuration

The `antenna_config` section defines antenna properties for each frequency.

```json
"antenna_config": {
  "700": {
    "center_frequency": 700,
    "target_power_mW": 267,
    "model_type": "PIFA",
    "source_name": "Lines 1",
    "materials": {...},
    "gridding": {...}
  }
}
```

**Key parameters**:

- `center_frequency`: Antenna operating frequency in MHz
- `target_power_mW`: The necessary power *such that* the 10g-SAR in a flat phantom is 1 W/kg. This is for a 300 x 300 mm x 5 skin depth region with tissue parameters from IEC/IEEE 62209-01 at a distance of 5 mm.
- `model_type`: Antenna type (PIFA or IFA), used to select setup logic
- `source_name`: Name of the excitation entity in the CAD file
- `materials`: Maps CAD component names to Sim4Life materials
- `gridding`: Defines the details of specific sub-meshes in the antenna, per entity group.

**Material mapping**

The `materials` object tells GOLIAT which material to assign to each antenna component:

```json
"materials": {
  "Extrude 1": "Copper",
  "component1:Battery": "Copper",
  "component1:Patch": "Copper",
  "component1:ShortingPin": "Copper",
  "component1:Substrate": "Rogers RT/duroid 5880"
}
```

These names must match entities in the CAD file. The substrate uses a specific dielectric material (Rogers RT/duroid 5880) while conductive parts use Copper.

![Antenna detail](../img/tutorials/tut3_antenna.png)

*PIFA antenna components grided with an automatic default grid. The antenna is below the top right circle. One can distinguish a battery, a substrate, ... This is a mock-up phone as is common in very general dosimetric studies.*

### Placement scenarios

The `placement_scenarios` section defines where and how the antenna is positioned relative to the phantom.

```json
"placement_scenarios": {
  "front_of_eyes": {
    "bounding_box": "default",
    "positions": {
      "center": [0, 0, -100]
    },
    "orientations": {
      "vertical": []
    },
    "antenna_reference": {
      "distance_from_top": 10
    },
    "phantom_reference": "nasion"
  }
}
```

**Key parameters**:

- `bounding_box`: Which part of phantom to include (default, head, trunk, whole_body). Default means head for `front_of_eyes` and trunk for `by_belly`.
- `positions`: Named position offsets in [x, y, z] mm relative to the reference point. GOLIAT will loop through all of these.
- `orientations`: Named rotation sequences applied to the antenna. GOLIAT will loop through all of these, *for each position*.
- `antenna_reference`: Point on antenna used for placement (distance from top of the phone, by the earpiece speaker).
- `phantom_reference`: Anatomical landmark used as placement origin.

**Anatomical landmarks**

The `phantom_reference` parameter points to a coordinate defined in `phantom_definitions`:

```json
"phantom_definitions": {
  "thelonious": {
    "head_y_separation": 0,
    "trunk_z_separation": -300,
    "chest_extension": 100,
    "nasion": [-1, 0, 0],
    ...
    "placements": {
      "do_front_of_eyes": true,
      ...
      "distance_from_eye": 200
    }
  }
}
```

For front_of_eyes placement:
- GOLIAT finds the eye entities in the phantom
- Calculates their bounding box center
- Applies the nasion offset
- Adds the position offset (here [0, 0, -100]) for the `center` scenario pacement.
- Places the antenna at the calculated point
- Orients the antenna according to any rotations specified for the scenario.

The `distance_from_eye` parameter (200 mm) sets the separation between the phantom and antenna along the Y axis (pointing out of the face).

The three parameters `head_y_separation`, `trunk_z_separation` and `chest_extension` define how the `head` and `trunk` bounding boxes are defined. Some dimensions of these bounding boxes are defined intelligently, others unfortuneatly require some manual parameters like these. For example, the `head_y_separation` determines where the back of the head stops for the Thelonious phantom. The next two determine the z and y dimensions of the trunk bounding box.

![Anatomical reference points](../img/tutorials/tut3_reference_points.png)

*Nasion point used as reference point for front_of_eyes placement. This aligns with a point on the antenna 10 mm below the top, by the earpiece speaker.*

![Setup for the near-field simulation](../img/tutorials/tut3_setup.png)

*Setup for the near-field simulation. Note that the head and trunk bounding boxes are present. The simulation bounding box wraps around the phone and the bounding box of interest (here 'default', i.e., the head). The phone is placed 100 mm below its initial position, where the naison aligns 200 mm from the antenna reference point, 10 mm below the top of the phone.*

---

## Understanding the GUI

Before running the simulation, let's tour the GOLIAT GUI. It shows real-time progress, logs, timings, and system utilization.

When you run a study, a window opens automatically. Here's what each section does:

![GUI overview](../img/tutorials/tut1_gui.gif)

### Main tab

The main tab shows overall progress and current activity.

**Progress bars**:
- Overall progress: Total completion across all simulations
- Stage progress: Current phase (setup, run, extract) within one simulation

**Simulation info**:
- Current simulation count (e.g., Simulation: 1 / 1000)
- Case details (phantom, frequency, placement)

**Timing info**:
- Elapsed time since study started
- Time remaining (estimated from profiler data)

**Status log**:
- Color-coded messages showing what GOLIAT is doing
- Green for success, yellow for warnings, red for errors

**System utilization**:
- CPU percentage across all cores
- RAM usage (used/total GB)
- GPU percentage (if `nvidia-smi` available)

**Buttons**:
- Hitting Stop will try to stop the simulations gracefully.
- Closing the window will instantly stop GOLIAT. This may corrupt your `.smash` files.
- You can minimize the GUI to the Windows system tray.

### Timings and Timings Piecharts tabs

Shows execution statistics for phases and subtasks.

For each task, you see:
- Mean, median, min, max duration
- 10th, 25th, 75th, 90th percentiles

This data comes from the profiler and improves ETA accuracy as more simulations complete. The piecharts give a visual depiction of the average timings.

### Time Remaining and Overall Progress tabs.

Shows progress and ETA trends over time.

**Time remaining plot**:
- Hours remaining vs time
- Should converge to zero as work completes

**Overall progress plot**:
- Percentage complete vs time
- Should show smooth upward trend

These plots update every 5 seconds during execution.

---

## The logging system

GOLIAT uses two separate log streams with different purposes.

### Log destinations

Logs go to three places:

1. **Console (shell)**: Real-time output while study runs
2. `logs/` **directory**: Session logs with timestamps (e.g., `logs/10_31_25-20-55-16.progress.log`)
3. **Simulation directories**: Simulation-specific logs in each project folder

### Progress vs verbose logs

**Progress log**:
- High-level, user-facing messages
- Shown in GUI status box
- Saved to `*.progress.log` files
- Tracks phases and subtasks

**Verbose log**:
- Detailed internal messages
- Not shown in GUI (console and file only)
- Saved to main `*.log` files
- Includes everything, including the Sim4Life and `iSolve` logs, useful for debugging

Both logs write to the console, but only progress logs appear in the GUI.

### Phases and subtasks

GOLIAT organizes work into three phases, each containing multiple subtasks.

**The three phases**:

1. **Setup**: Creates Sim4Life project, imports models, configures simulation
2. **Run**: Executes FDTD solver (longest phase)
3. **Extract**: Processes SAR data, generates reports

**Subtask logging**:

When a subtask starts, you see:
```
  - Loading phantom model...
```

When it finishes, you see:
```
    - Done in 3.2s
```

The timing line is indented to align with the subtask name. This format applies to console, GUI, and log files. A phase (containing subtasks) is also timed and aligned one tab less.

![Logging example](../img/tutorials/tut3_logging_example.png)

*Console output showing subtask progression with timings.*

### Simulation-specific logs

Each simulation writes logs to its project directory:

```
results/near_field/thelonious/700MHz/front_of_eyes_center_vertical/
    progress.log        # High-level progress
    verbose.log         # Detailed logs
    config.json         # Metadata for caching
    simulation.smash    # Sim4Life project
```

The `progress.log` file is a subset of `verbose.log`, containing only user-facing messages. This is the log shown in the GUI.

---

## Running the simulation

Now that we finally understand all the above, it's time to run the near-field study:

In [ ]:
run_bash("goliat study tutorial_3_near_field")

Running: source .bashrc && goliat study tutorial_3_near_field

------------------------------------------------------------
Starting Sim4Life application... 
Initializing Application [stdout]
Initializing Application [stderr]
[Warn]  Unable to load module 'C:\Program Files\Sim4Life_8.2.0.16876\MusaikInterface.xdll'
[Info]  Connection to local Ares successfully established.
Sim4Life application started. 
--- Starting Near-Field Study: tutorial_3_near_field.json --- [NearFieldStudy._run_study]

--- Processing Simulation 1/1: thelonious, 700MHz, front_of_eyes_center_vertical --- [NearFieldStudy._run_study]
--- Starting: setup --- [profile]
Project path set to: C:/Users/user/repo-clean/results/near_field/thelonious/700MHz/front_of_eyes_center_vertical/near_field_thelonious_700MHz_front_of_eyes_center_vertical.smash [ProjectManager.create_or_open_project]
No metadata file found at config.json. [ProjectManager.verify_simulation_metadata]
Existing project is invalid or out of date. A new setu

0



The GUI opens and shows progress through three phases:

**Setup phase** (1-2 minutes):
- Create project directory
- Import phantom model (thelonious)
- Import antenna CAD model
- Place antenna at nasion landmark, and lower this by 100 mm.
- Apply material properties
- Configure simulation boundaries
- Set up gridding

**Run phase** (anwyhere between 1 minute and 5 hours depending on simulation settings):
- Writes the input file
- Launch iSolve and executes the FDTD simulation
- Monitors convergence

**Extract phase** (1-2 minutes):
- Open completed project
- Extract SAR data from sensors
- Generate statistics reports
- Save deliverables

Watch the GUI status log to see each subtask complete with its timing.

---

## Understanding the results

After the simulation completes, find the results in:

```
results/near_field/thelonious/700MHz/
  front_of_eyes_center_vertical/
```

### Output files

**Simulation project**:
- `simulation.smash`: Complete Sim4Life project with fields and SAR

**SAR data**:
- `sar_results.json`: Raw SAR values for all tissues
- `sar_stats_all_tissues.pkl`: Statistical summaries (pickle format)
- `sar_stats_all_tissues.html`: Human-readable HTML report

**Metadata**:
- `config.json`: Config snapshot and caching flags

Analysis and visualization sections will be added in a future tutorial.

---

## Checking the Sim4Life project

Open the project to inspect the simulation setup:

In [ ]:
run_bash("ls results/near_field/thelonious/700MHz/front_of_eyes_center_vertical/")

Running: source .bashrc && ls results/near_field/thelonious/700MHz/front_of_eyes_center_vertical/

------------------------------------------------------------
config.json
near_field_thelonious_700MHz_front_of_eyes_center_vertical.smash
near_field_thelonious_700MHz_front_of_eyes_center_vertical.smash_Results
point_sensor_data.png
progress.log
sar_results.json
sar_stats_all_tissues.html
sar_stats_all_tissues.pkl
verbose.log
------------------------------------------------------------

Command completed with return code: 0


0



You can open this `.smash` file in Sim4Life to see:
- Phantom geometry with tissue materials
- Antenna positioned in front of eyes
- Simulation bounding box (head region)
- FDTD grid resolution
- Point sensors for SAR measurement

---

## What's next

You've completed your first near-field simulation. Next steps:

**Tutorial 4**: Gridding and phantom rotation
- Manual vs automatic gridding
- Subgridding for antenna components
- By_cheek placement with phantom rotation
- Scene alignment optimization

**Tutorial 5**: Parallel and cloud execution
- Running multiple simulations simultaneously
- Config splitting strategies
- Using oSPARC for cloud compute

---

## Summary

You learned:
- Near-field simulations model device exposure with antennas near the body
- Antenna configuration includes materials, power, and gridding settings
- Placement scenarios use anatomical landmarks for positioning
- The GUI shows real-time progress, logs, timings, and system utilization
- GOLIAT uses two log streams (progress and verbose) with different purposes
- Simulations progress through setup, run, and extract phases
- Results include SAR data, statistics, and complete Sim4Life projects

In Tutorial 4, we'll explore computational gridding strategies and more complex placements requiring phantom rotation.